<a href="https://colab.research.google.com/github/Malthr/sembago-ml/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project ID : CH2-PS393

Project Name: Sembago

Project Themes: Food Accessibility, Agribusiness, and Food Security


Project Team Members:

(ML) M324BSY1271 - Syukri Mar-Am - Universitas Tanjungpura

(ML) M359BSY1158 - Mohammad Alif Tuharea - Universitas Palangka Raya

(ML) M131BSX1924 - Nevi Marita Rimadhani - Politeknik Negeri Malang

(CC) C010BSY3251 - Bintang Satrya Leksono - Universitas Indonesia

(CC) C568BKY4423 - Insanu Sholeh Jum Gunawan - Universitas Islam Negeri Walisongo Semarang

(MD) A568BKX4455 - Siti Zikrina Farahdiba - Universitas Islam Negeri Walisongo Semarang

(MD) A248BSY2902 - Farhan Aditya - Universitas Lampung

In [56]:
#Import Package

import pandas as pd
import numpy as np
import pprint
import os

In [2]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/Shareddrives/Machine Learning CH2-PS393


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/Machine Learning CH2-PS393


In [3]:
df_rating = pd.read_csv('dummy_ratings.csv')
display(df_rating)

,id_user,id_produk,id_market,rating
0,b680,a074,b595,4
1,b759,a248,b851,5
2,c196,a356,c044,3
3,c288,a519,c221,1
4,c600,a612,g864,3
...,...,...,...,...
495,z204,z327,w285,5
496,z303,z420,x399,5
497,z360,z663,y554,4
498,z584,z689,z840,2


In [4]:
df_user = pd.read_csv('dummy_produk.csv')
display(df_user)

,id_produk,id_market,nama_produk,stok
0,a074,s685,Bruen,37
1,a248,j668,Bernhard,42
2,a356,n829,Effertz,47
3,a519,s408,Shields,38
4,a612,s926,White,21
...,...,...,...,...
245,z327,h995,Jaskolski,48
246,z420,g864,Goodwin,29
247,z663,h995,Bergnaum,28
248,z689,b595,Rodriguez,37


In [5]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_datasets as tfds

In [6]:
# Convert CSV Dataset into TensorFlow Dataset
def load_dataset(file_path, features, labels=None):
  df = pd.read_csv(file_path)

  features = df[features]

  if labels:
    labels = df[labels]
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  else:
    dataset = tf.data.Dataset.from_tensor_slices(dict(features))

  return dataset

In [7]:
ratings = load_dataset('dummy_ratings.csv', ['id_user', 'id_produk', 'id_market'])
produks = load_dataset('dummy_produk.csv', ['id_produk', 'id_market'])

In [8]:
for x in produks.take(1):
  pprint.pprint(x)

{'id_market': <tf.Tensor: shape=(), dtype=string, numpy=b's685'>,
 'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>}


In [9]:
for x in ratings.take(1):
  pprint.pprint(x)

{'id_market': <tf.Tensor: shape=(), dtype=string, numpy=b'b595'>,
 'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>,
 'id_user': <tf.Tensor: shape=(), dtype=string, numpy=b'b680'>}


In [10]:
rating_size = len(list(ratings))

train_size = int(0.7 * rating_size)
val_size = int(0.15 * rating_size)
test_size = rating_size - train_size - val_size

ratings_train = ratings.take(train_size)
ratings_temp = ratings.skip(train_size)
ratings_val = ratings_temp.take(val_size)
ratings_test = ratings_temp.skip(val_size)

In [11]:
produks_size = len(list(produks))

train_size = int(0.7 * produks_size)
val_size = int(0.15 * produks_size)
test_size = produks_size - train_size - val_size

produks_train = produks.take(train_size)
produks_temp = produks.skip(train_size)
produks_val = produks_temp.take(val_size)
produks_test = produks_temp.skip(val_size)

In [12]:
for x in ratings.take(1):
  pprint.pprint(x)

{'id_market': <tf.Tensor: shape=(), dtype=string, numpy=b'b595'>,
 'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>,
 'id_user': <tf.Tensor: shape=(), dtype=string, numpy=b'b680'>}


In [13]:
for x in produks.take(1):
  pprint.pprint(x)

{'id_market': <tf.Tensor: shape=(), dtype=string, numpy=b's685'>,
 'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>}


In [14]:
sample_element = next(iter(produks_train))
sample_element_keys = list(sample_element.keys())

print("Key names in the dataset:", sample_element_keys)

sample_element = next(iter(ratings_train))
sample_element_keys = list(sample_element.keys())

print("Key names in the dataset:", sample_element_keys)

Key names in the dataset: ['id_produk', 'id_market']
Key names in the dataset: ['id_user', 'id_produk', 'id_market']


In [15]:
ratings = ratings.map(lambda x: {
    "id_user": x["id_user"],
    "id_produk": x["id_produk"],
})

produks = produks.map(lambda x: {
    "id_produk": x["id_produk"],
})

In [16]:
for x in ratings.take(1):
  pprint.pprint(x)

{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>,
 'id_user': <tf.Tensor: shape=(), dtype=string, numpy=b'b680'>}


In [17]:
for x in produks.take(1):
  pprint.pprint(x)

{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>}


In [18]:
print(len(produks))

250


In [19]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(1_000, seed=42, reshuffle_each_iteration=False)

train_percentage = 80

train_size = int(train_percentage / 100 * len(ratings))

train = shuffled.take(train_size)
test = shuffled.skip(train_size).take(200)

In [20]:
print(len(list(test)))

100


In [21]:
for x in produks.take(5).as_numpy_iterator():
  pprint.pprint(x)

{'id_produk': b'a074'}
{'id_produk': b'a248'}
{'id_produk': b'a356'}
{'id_produk': b'a519'}
{'id_produk': b'a612'}


In [22]:
produks_batched = produks.batch(100).map(lambda x: x["id_produk"])
ratings_batched = ratings.batch(300).map(lambda x: x["id_user"])

In [23]:
for x in produks_batched.take(1):
    print(x)

tf.Tensor(
[b'a074' b'a248' b'a356' b'a519' b'a612' b'a680' b'a937' b'a966' b'b008'
 b'b194' b'b248' b'b267' b'b277' b'b352' b'b444' b'b524' b'b538' b'b548'
 b'b711' b'b800' b'b851' b'b866' b'b889' b'b999' b'c001' b'c215' b'c459'
 b'c466' b'c513' b'c527' b'c572' b'c582' b'c733' b'c980' b'd011' b'd181'
 b'd208' b'd313' b'd355' b'd378' b'd411' b'd696' b'd735' b'd938' b'e090'
 b'e112' b'e439' b'e488' b'e720' b'e799' b'e826' b'e837' b'e908' b'f003'
 b'f024' b'f116' b'f139' b'f238' b'f243' b'f247' b'f399' b'f494' b'f686'
 b'f770' b'f861' b'f867' b'f902' b'g124' b'g134' b'g269' b'g332' b'g517'
 b'g652' b'g786' b'g857' b'g875' b'g960' b'h101' b'h299' b'h367' b'h652'
 b'h667' b'i146' b'i239' b'i276' b'i294' b'i409' b'i581' b'i848' b'i870'
 b'i973' b'j004' b'j065' b'j066' b'j200' b'j485' b'j614' b'j723' b'j787'
 b'j810'], shape=(100,), dtype=string)


In [24]:
for x in ratings_batched.take(1):
  pprint.pprint(x)

<tf.Tensor: shape=(300,), dtype=string, numpy=
array([b'b680', b'b759', b'c196', b'c288', b'c600', b'c627', b'c662',
       b'c723', b'c797', b'd314', b'd907', b'e156', b'e440', b'e721',
       b'e973', b'f201', b'f255', b'f567', b'f734', b'f804', b'g006',
       b'g035', b'g124', b'g971', b'h156', b'h868', b'i266', b'i375',
       b'i620', b'i744', b'i838', b'j210', b'j320', b'j504', b'k393',
       b'k760', b'l005', b'l403', b'l687', b'l698', b'm172', b'm733',
       b'n066', b'n751', b'n845', b'o512', b'o541', b'o755', b'p135',
       b'p414', b'p620', b'p985', b'q050', b'q174', b'q263', b'q369',
       b'q370', b'q413', b'q422', b'q612', b'q702', b'q823', b'q970',
       b'r138', b'r871', b'r986', b's715', b's993', b't085', b't111',
       b't132', b't204', b't300', b't786', b'u199', b'u806', b'v541',
       b'v564', b'v613', b'v623', b'v916', b'v946', b'v948', b'w601',
       b'w612', b'w754', b'x231', b'x385', b'x999', b'y005', b'y328',
       b'y704', b'y947', b'z022', b'z165', 

In [25]:
unique_id_users = np.unique(np.concatenate(list(ratings_batched)))
unique_id_produks = np.unique(np.concatenate(list(produks_batched)))

In [26]:
unique_id_users = np.unique([entry['id_user'].numpy() for entry in ratings]).astype(object)
unique_id_produks = np.unique([entry['id_produk'].numpy() for entry in produks]).astype(object)

In [27]:
unique_id_users[:10]

array([b'b680', b'b759', b'c196', b'c288', b'c600', b'c627', b'c662',
       b'c723', b'c797', b'd314'], dtype=object)

In [40]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_id_users, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_id_users) + 1, 64)
])

In [41]:
produk_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_id_produks, mask_token=None),
    tf.keras.layers.Embedding(len(unique_id_produks) + 1, 64)
])

In [30]:
for x in produks.take(5):
    print(x)

{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a074'>}
{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a248'>}
{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a356'>}
{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a519'>}
{'id_produk': <tf.Tensor: shape=(), dtype=string, numpy=b'a612'>}


In [42]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=produks_batched.map(produk_model)
)

In [43]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [44]:
from typing import Dict, Text

class ProdukRecModel(tfrs.Model):

  def __init__(self, user_model, produk_model, task):
    super().__init__()
    self.user_model: tf.keras.Model = user_model
    self.produk_model: tf.keras.Model = produk_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["id_user"])
    # And pick out the produk features and pass them into the produk model,
    # getting embeddings back.
    positive_produk_embeddings = self.produk_model(features["id_produk"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_produk_embeddings)


In [45]:
class NoBaseClassProdukRecModel(tf.keras.Model):

  def __init__(self, user_model, produk_model, task):
    super().__init__()
    self.user_model: tf.keras.Model = user_model
    self.produk_model: tf.keras.Model = produk_model
    self.task: tf.keras.layers.Layer = task

  def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # Set up a gradient tape to record gradients.
    with tf.GradientTape() as tape:
      # Loss computation.
      user_embeddings = self.user_model(features["id_user"])
      positive_produk_embeddings = self.produk_model(features["id_produk"])
      loss = self.task(user_embeddings, positive_produk_embeddings)

      # Handle regularization losses as well.
      regularization_loss = sum(self.losses)

      total_loss = loss + regularization_loss

    gradients = tape.gradient(total_loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

  def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # Loss computation.
    user_embeddings = self.user_model(features["id_user"])
    positive_produk_embeddings = self.produk_model(features["id_produk"])
    loss = self.task(user_embeddings, positive_produk_embeddings)

    # Handle regularization losses as well.
    regularization_loss = sum(self.losses)

    total_loss = loss + regularization_loss

    metrics = {metric.name: metric.result() for metric in self.metrics}
    metrics["loss"] = loss
    metrics["regularization_loss"] = regularization_loss
    metrics["total_loss"] = total_loss

    return metrics

In [51]:
model = ProdukRecModel(user_model, produk_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.0001))

In [48]:
cached_train = train.shuffle(100).batch(8192).cache()
cached_test = test.batch(100).cache()

In [52]:
model.fit(cached_train, epochs=250)

Epoch 1/250
1/1 [==============================] - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9225 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 2338.5972 - regularization_loss: 0.0000e+00 - total_loss: 2338.5972
Epoch 2/250
1/1 [==============================] - 0s 78ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.9225 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 2338.5068 - regularization_loss: 0.0000e+00 - total_loss: 2338.5068
Epoch 3/250
1/1 [==============================] - 0s 80ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accurac

In [53]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 468ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.3100 - factorized_top_k/top_10_categorical_accuracy: 0.5600 - factorized_top_k/top_50_categorical_accuracy: 0.7100 - factorized_top_k/top_100_categorical_accuracy: 0.7900 - loss: 455.8046 - regularization_loss: 0.0000e+00 - total_loss: 455.8046


{'factorized_top_k/top_1_categorical_accuracy': 0.0,
 'factorized_top_k/top_5_categorical_accuracy': 0.3100000023841858,
 'factorized_top_k/top_10_categorical_accuracy': 0.5600000023841858,
 'factorized_top_k/top_50_categorical_accuracy': 0.7099999785423279,
 'factorized_top_k/top_100_categorical_accuracy': 0.7900000214576721,
 'loss': 455.8045654296875,
 'regularization_loss': 0,
 'total_loss': 455.8045654296875}

In [54]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

In [59]:
# Export the query model.
path = "retrieval_model"

# Save the index.
tf.saved_model.save(index, path)